In [1]:
%run Cisco\ Common.ipynb

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [2]:
from IPython.core.display import display, HTML
import re

def parse_brief(cell):
    ids = cell.xpath(".//b|.//strong")
    result = {}
    if ids:
        result["id"] = parse_text(ids[0])
    links = cell.xpath(".//a")
    if links:
        result["link"] = links[0].attrib["href"]
        result["description"] = parse_text_content(links[0].xpath("text()"))
    else:
        result["description"] = parse_text(cell)

    return result


def parse_table_title(table):
    return " ".join(
        map(parse_text, table.xpath("ancestor::div[@class='table parbase section']/preceding-sibling::div[1]//h2")))


def parse_acrobat_feed(feed):
    cell_parsers = {
        "Brief": parse_brief,
        "Originally posted": parse_date,
        "Last updated": parse_date
    }
    return {
        parse_table_title(t): sorted(parse_table(t, cell_parsers), key=lambda r: r["Last updated"], reverse=True)
        for t in
        feed.xpath("//div[@id='main-content']//table")
    }

feed = parse_acrobat_feed(get_html("https://helpx.adobe.com/security/products/acrobat.html"))
url = next(iter(feed.values()))[0]['Brief']['link']
content = requests.get(url).text
info_content = re.findall('<table width="100%" cellspacing="0" cellpadding="1" border="1">(.*)</table>', content,re.MULTILINE|re.DOTALL)[0]

display(HTML("<h1>Adobe Acrobat Latest Vulnerability</h1>"))
print(url)
display(HTML("<table>"+info_content+"</table></div>"))

https://helpx.adobe.com/security/products/acrobat/apsb19-55.html


In [3]:
#AMD
#too much js



In [4]:
from IPython.core.display import display, HTML
from datetime import date 

# month = date.today().strftime("%Y+%b") 
# I commented the one above, since there are no vulnerabilities for 2020+Jan
month = "2020+Jan"

display(HTML("<h1>Cisco AnyConnect Monthly Vulnerability Feed</h1>"))
display(HTML(get_cisco_vuln_table(get_cisco_vuln_data(109810, month))))

Identifer,Severity,Title,ciscoBugId,Last Published
cisco-sa-20200108-anyconnect-hijack,Medium,Cisco AnyConnect Secure Mobility Client for Android Service Hijack Vulnerability,CSCvr67149,2020-01-08


In [5]:
date_today = date.today().strftime("%m/%d/%Y")
url = "https://portal.msrc.microsoft.com/api/security-guidance/en-us/summary?filterText=edge&fromPublishedDate=12%2F11%2F2019&isDescending=true&orderBy=publishedDate&pageNumber=1&pageSize=20&toPublishedDate="+date_today

data = get_json(url)

display(HTML(table(
    data['details'],
    [
        {
            "key": "publishedDate", 
            "title": "Date", 
            "processor": lambda x: x[:10]
        },
        {
            "key": "updateSoftwareList", 
            "title": "Software", 
            "processor": lambda x: x[0]
        },
        {
            "key": "articleUrl1", 
            "title": "Urls", 
            "processor": lambda x: f'<a href="{x}">{x}</a>'
        },
    ]
)))

Date,Software,Urls
2020-01-28,Microsoft Edge (Chromium-based),https://docs.microsoft.com/en-us/DeployEdge/microsoft-edge-relnotes-security


In [6]:
from urllib.parse import urljoin

url = "https://support.google.com/chrome/a/answer/7679408/"

row = get_html(url).xpath("//div[@class='article-content-container']/*/table//tr[2]/td")
version = row[0].xpath("a/text()")[0]
link = urljoin(url, row[1].xpath("a/@href")[0])

display(HTML("<h1>Google Chrome Latest Version</h1>"))
display(HTML(f'{version} <a href="{link}">Release PDF</a>'))

In [7]:
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor

url = "https://www.intel.com/content/www/us/en/security-center/default.html"

def parse_advisory(cell):
    link = cell.xpath("a")[0]
    return {
        "link": urljoin(url, link.attrib['href']),
        "text": link.text
    }

advisories = parse_table(
    lxml.html.fromstring(
        requests.get(url).content
    ).xpath("//table")[0],
    parsers = {"Advisories": parse_advisory}
)

def get_advisory_details(url):
    return {
        "url": url,
        **{
            parse_text(row[0]).rstrip(":").strip():parse_text(row[1]) 
            for row in 
            get_html(url).xpath("//table")[0].xpath(".//tr")
        }
    }

with ThreadPoolExecutor(max_workers=5) as executor:
    advisory_details = executor.map(get_advisory_details, (a["Advisories"]["link"] for a in advisories[:1]))

display(HTML(table(
    advisory_details,
    [
        {
            "key": None, 
            "title": "URL", 
            "processor": lambda r: f'<a href="{r["url"]}">{r["Intel ID"]}</a>'
        },
        {
            "key": "Severity rating", 
            "title": "Severity", 
        },
        {
            "key": "Last revised", 
            "title": "Last Date", 
        },
    ]
)))

URL,Severity,Last Date
INTEL-SA-00329,MEDIUM,01/27/2020


In [14]:
from datetime import date

# today = date.today().strftime("%b%Y")
today = "Jan2020"
url = f"https://www.oracle.com/security-alerts/cpu{today.lower()}.html"

doc = get_html(url)

oracle_risk_matrices = {
    parse_text(i.xpath("preceding-sibling::h4[1]")[0]):
    parse_table(i.xpath(".//table[@class='otable-w2']")[0]) 
    for i in 
    doc.xpath("//div[@data-ocomid='otable'][.//table[@class='otable-w2']]")
}

def flatten_cvss(matrix):
    return [
        {
            **{k:v for k, v in i.items() if k != "CVSS VERSION 3.0 RISK (see Risk Matrix Definitions )"},
            **{f"CVSS {k}":v for k, v in i["CVSS VERSION 3.0 RISK (see Risk Matrix Definitions )"].items()}
        }
        for i in 
        matrix
    ]

display(HTML("<h1>Java "+today+" Vulnerabilites</h1>"))
print(url)

interesting_risk_matrices = {f"Oracle {i} Risk Matrix" for i in ["Java SE", "Database Server", "MySQL"]}

for title, risk_matrix in [(k, flatten_cvss(v)) for k, v in oracle_risk_matrices.items() if k in interesting_risk_matrices]:
    display(HTML("<h2>Affected Versions</h2>"))
    print(", ".join(sorted(set(chain.from_iterable(map(str.strip, r['Supported Versions Affected'].split(",")) for r in risk_matrix)))))
    display(HTML(f"<h3>{title}</h3>"))
    display(HTML(table(risk_matrix)))

https://www.oracle.com/security-alerts/cpujan2020.html


11.2.0.4, 12.1.0.2, 12.2.0.1, 18c, 19c


Protocol,CVSS Base Score,Component,CVSS Scope,CVSS Confid- entiality,CVSS Attack Complex,Remote Exploit without Auth.?,CVSS Attack Vector,CVSS Privs Req'd,CVSS Avail- ability,Package and/or Privilege Required,CVE#,Notes,CVSS User Interact,CVSS Inte- grity,Supported Versions Affected
OracleNet,7.7,Core RDBMS,Changed,None,Low,No,Network,Low,High,Create Session,CVE-2020-2511,,None,None,"12.1.0.2, 12.2.0.1, 18c, 19c"
OracleNet,7.5,Core RDBMS,Un- changed,High,High,Yes,Network,None,High,None,CVE-2020-2510,,Required,High,"11.2.0.4, 12.1.0.2, 12.2.0.1, 18c, 19c"
Multiple,7.5,Java VM,Un- changed,High,High,No,Network,Low,High,Create Session,CVE-2020-2518,,None,High,"11.2.0.4, 12.1.0.2, 12.2.0.1, 18c, 19c"
HTTP,7.5,Workload Manager (Apache Tomcat),Un- changed,None,Low,Yes,Network,None,High,None,CVE-2019-10072,See Note 1,None,None,"12.2.0.1, 18c, 19c"
OracleNet,5.9,Database Gateway for ODBC,Un- changed,None,High,Yes,Network,None,High,None,CVE-2020-2512,,None,None,"11.2.0.4, 12.1.0.2, 12.2.0.1, 18c, 19c"
OracleNet,5.0,Database Gateway for ODBC,Un- changed,Low,High,No,Network,Low,Low,Create Session,CVE-2020-2515,,None,Low,"11.2.0.4, 12.1.0.2, 12.2.0.1, 18c, 19c"
OracleNet,4.1,Core RDBMS,Changed,Low,Low,No,Network,High,None,"Create Index, Create Table",CVE-2020-2527,,None,None,"12.1.0.2, 12.2.0.1, 18c, 19c"
Local Logon,3.9,Core RDBMS,Un- changed,None,Low,No,Local,Low,Low,Local Logon,CVE-2020-2731,,Required,Low,"12.1.0.2, 12.2.0.1, 18c, 19c"
Local Logon,3.9,Oracle Applications DBA,Un- changed,None,Low,No,Local,Low,Low,Local Logon,CVE-2020-2568,,Required,Low,"12.1.0.2, 12.2.0.1, 18c, 19c"
Local Logon,3.9,Oracle Applications DBA,Un- changed,None,Low,No,Local,Low,Low,Local Logon,CVE-2020-2569,,Required,Low,"12.2.0.1, 18c, 19c"


11.0.5, 13.0.1, 13.0.1; Java SE Embedded: 8u231, 8u231, 8u231; Java SE Embedded: 8u231, Java SE: 11.0.5, Java SE: 7u241, Java SE: 8u231


Protocol,CVSS Base Score,Component,CVSS Scope,CVSS Confid- entiality,CVSS Attack Complex,Remote Exploit without Auth.?,Product,CVSS Attack Vector,CVSS Privs Req'd,CVSS Avail- ability,CVSS Inte- grity,CVE#,Notes,CVSS User Interact,Supported Versions Affected
Multiple,8.1,Serialization,Un- changed,High,High,Yes,"Java SE, Java SE Embedded",Network,None,High,High,CVE-2020-2604,See Note 1,None,"Java SE: 7u241, 8u231, 11.0.5, 13.0.1; Java SE Embedded: 8u231"
Multiple,7.5,JavaFX (SQLite),Un- changed,None,Low,Yes,Java SE,Network,None,High,None,CVE-2019-16168,See Note 2,None,Java SE: 8u231
Multiple,7.5,JavaFX (libxslt),Un- changed,High,Low,Yes,Java SE,Network,None,None,None,CVE-2019-13117,See Note 2,None,Java SE: 8u231
Multiple,7.5,JavaFX (libxslt),Un- changed,High,Low,Yes,Java SE,Network,None,None,None,CVE-2019-13118,See Note 2,None,Java SE: 8u231
Kerberos,6.8,Security,Changed,High,High,Yes,"Java SE, Java SE Embedded",Network,None,None,None,CVE-2020-2601,See Note 1,None,"Java SE: 7u241, 8u231, 11.0.5, 13.0.1; Java SE Embedded: 8u231"
Multiple,5.9,JavaFX,Un- changed,None,High,Yes,Java SE,Network,None,None,High,CVE-2020-2585,See Note 1,None,Java SE: 8u231
HTTPS,4.8,JSSE,Un- changed,Low,High,Yes,Java SE,Network,None,None,Low,CVE-2020-2655,See Note 1,None,"Java SE: 11.0.5, 13.0.1"
Multiple,4.8,Networking,Un- changed,Low,High,Yes,"Java SE, Java SE Embedded",Network,None,None,Low,CVE-2020-2593,See Note 1,None,"Java SE: 7u241, 8u231, 11.0.5, 13.0.1; Java SE Embedded: 8u231"
Multiple,3.7,Libraries,Un- changed,None,High,Yes,Java SE,Network,None,Low,None,CVE-2020-2654,See Note 3,None,"Java SE: 7u241, 8u231, 11.0.5, 13.0.1"
Kerberos,3.7,Security,Un- changed,None,High,Yes,"Java SE, Java SE Embedded",Network,None,None,Low,CVE-2020-2590,See Note 1,None,"Java SE: 7u241, 8u231, 11.0.5, 13.0.1; Java SE Embedded: 8u231"


3.12.4 and prior, 4.1.3 and prior, 5.3.13 and prior, 5.6.46 and prior, 5.7.28 and prior, 7.3.27 and prior, 7.4.25 and prior, 7.5.15 and prior, 7.6.12 and prior, 8.0.17 and prior, 8.0.18 and prior


Protocol,CVSS Base Score,Component,CVSS Scope,CVSS Confid- entiality,CVSS Attack Complex,Remote Exploit without Auth.?,Product,CVSS Attack Vector,CVSS Privs Req'd,CVSS Avail- ability,CVSS Inte- grity,CVE#,Notes,CVSS User Interact,Supported Versions Affected
MySQL Workbench,7.5,MySQL Workbench (SQLite),Un- changed,None,Low,Yes,MySQL Workbench,Network,None,High,None,CVE-2019-16168,,None,8.0.18 and prior
TLS,7.4,Connector/ODBC (OpenSSL),Un- changed,High,High,Yes,MySQL Connectors,Network,None,None,High,CVE-2019-1547,,None,"5.3.13 and prior, 8.0.18 and prior"
MySQL Protocol,6.5,Server: Optimizer,Un- changed,None,Low,No,MySQL Server,Network,Low,High,None,CVE-2020-2579,,None,"5.6.46 and prior, 5.7.28 and prior, 8.0.18 and prior"
MySQL Protocol,6.5,Server: Optimizer,Un- changed,None,Low,No,MySQL Server,Network,Low,High,None,CVE-2020-2686,,None,8.0.18 and prior
MySQL Protocol,6.5,Server: Parser,Un- changed,None,Low,No,MySQL Server,Network,Low,High,None,CVE-2020-2627,,None,8.0.18 and prior
MySQL Protocol,5.9,C API,Un- changed,None,High,Yes,MySQL Client,Network,None,High,None,CVE-2020-2570,,None,"5.7.28 and prior, 8.0.18 and prior"
MySQL Protocol,5.9,C API,Un- changed,None,High,Yes,MySQL Client,Network,None,High,None,CVE-2020-2573,,None,"5.7.28 and prior, 8.0.18 and prior"
MySQL Protocol,5.9,C API,Un- changed,None,High,Yes,MySQL Client,Network,None,High,None,CVE-2020-2574,,None,"5.6.46 and prior, 5.7.28 and prior, 8.0.18 and prior"
MySQL Protocol,4.9,InnoDB,Un- changed,None,Low,No,MySQL Server,Network,High,High,None,CVE-2020-2577,,None,"5.7.28 and prior, 8.0.18 and prior"
MySQL Protocol,4.9,InnoDB,Un- changed,None,Low,No,MySQL Server,Network,High,High,None,CVE-2020-2589,,None,"5.7.28 and prior, 8.0.17 and prior"


In [9]:
import requests
import lxml.etree 

url = "https://support.lenovo.com/gb/en/api/v4/search/psrss?language=en&country=gb&brand=EBG,TPG,IPG,MOTO,PHONE,TABLET,SMART"

items = [{f.tag:f.text for f in i} for i in get_html(url).xpath("/rss/channel/item")]

display(HTML("<h1>Lenovo Latest Vulnerabilities</h1>"))
display(HTML(table(
    [i for i in items if "PS" in i["link"]][:5],
    [
        {
            "key": "link", 
            "title": "Ref", 
            "processor": lambda x: f'<a href=\"{x}\">{x.split("/")[-1]}</a>'
        },
        {
            "key": "title", 
            "title": "Title", 
        },
        {
            "key": "pubDate", 
            "title": "Date", 
            "processor": lambda x: x[:16]
        },
    ]
)))

Ref,Title,Date
